In [ ]:
pip install torch transformers datasets sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import json
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
from sentence_transformers import SentenceTransformer, util
import pickle

In [ ]:
# Load dataset
with open("datasetDL.json", "r") as file:
    data = json.load(file)["intents"]

# Flatten dataset: pertanyaan dan respons
questions = []
responses = []
for intent in data:
    questions.extend(intent["text"])
    responses.extend(intent["responses"])

# Load model Sentence Transformer
model = SentenceTransformer("distiluse-base-multilingual-cased-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [ ]:
# Menyimpan model
model.save("./model")

#Simpan model PKL
with open("HivMate_sentence_transformer_model.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
# Encode semua pertanyaan di dataset
question_embeddings = model.encode(questions, convert_to_tensor=True)

# Data uji: Pertanyaan dan jawaban tentang HIV
test_data = [
    {"input": "Apa itu HIV?", "expected_response": "HIV adalah virus yang menyerang sistem kekebalan tubuh."},
    {"input": "Bagaimana cara penularan HIV?", "expected_response": "HIV dapat menular melalui kontak dengan darah, hubungan seksual, atau dari ibu ke anak selama kehamilan."},
    {"input": "Apa gejala awal HIV?", "expected_response": "Gejala awal HIV mirip dengan flu, seperti demam, sakit kepala, dan ruam kulit."},
    {"input": "Bisakah HIV disembuhkan?", "expected_response": "Saat ini belum ada obat untuk menyembuhkan HIV, tetapi pengobatan antiretroviral (ARV) dapat mengontrol virus."},
    {"input": "Bagaimana cara mencegah HIV?", "expected_response": "Menggunakan kondom, tidak berbagi jarum suntik, dan melakukan tes HIV secara rutin dapat mencegah penularan HIV."},

]

In [ ]:
# Evaluasi
correct_predictions = 0
threshold = 0.6  # Menurunkan threshold
all_similarities = []

y_true = []
y_pred = []

for item in test_data:
    user_input = item["input"]
    expected_response = item["expected_response"]

    # Encode user input
    user_embedding = model.encode(user_input, convert_to_tensor=True)

    # Hitung kesamaan kosinus
    similarities = util.cos_sim(user_embedding, question_embeddings)
    max_idx = similarities.argmax().item()
    max_similarity = similarities[0, max_idx].item()
    all_similarities.append(max_similarity)

    predicted_response = responses[max_idx]

    print(f"Pertanyaan: {user_input}")
    print(f"Prediksi: {predicted_response}")
    print(f"Jawaban yang Diharapkan: {expected_response}")
    print(f"Similarity: {max_similarity:.2f}\n")

    if max_similarity > threshold:
        correct_predictions += 1
        y_true.append(1)
        y_pred.append(1)
    else:
        y_true.append(1)
        y_pred.append(0)

Pertanyaan: Apa itu HIV?
Prediksi: Singkatan dari Human Immunodeficiency Virus, sebuah virus yang menyerang sistem kekebalan tubuh manusia.
Jawaban yang Diharapkan: HIV adalah virus yang menyerang sistem kekebalan tubuh.
Similarity: 1.00

Pertanyaan: Bagaimana cara penularan HIV?
Prediksi: Hubungan Seks Tanpa Alat Pengaman, Berbagi Alat Suntik, Ibu Hamil ke Bayi, Transfusi Darah, Seks Oral, Pertukaran Cairan Vagina atau Sperma, Ibu yang terinfeksi HIV dan menyusui dapat menularkan virus kepada bayinya melalui ASI.
Jawaban yang Diharapkan: HIV dapat menular melalui kontak dengan darah, hubungan seksual, atau dari ibu ke anak selama kehamilan.
Similarity: 0.95

Pertanyaan: Apa gejala awal HIV?
Prediksi: Tanda-tanda awal infeksi HIV meliputi demam yang disertai ruam makulopapular, ulserasi mukosa (di mulut, anus, atau penis), sindrom retroviral akut yang menyerupai flu, serta gejala neurologis seperti meningitis aseptik.
Jawaban yang Diharapkan: Gejala awal HIV mirip dengan flu, seperti d

In [ ]:
# Menghitung akurasi
accuracy = correct_predictions / len(test_data)
print(f"Akurasi: {accuracy:.2f}")

# Menghitung rata-rata kemiripan
average_similarity = np.mean(all_similarities)
print(f"Rata-rata Similarity: {average_similarity:.2f}")

# Menghitung Precision, Recall, dan F1-Score
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

Akurasi: 1.00
Rata-rata Similarity: 0.92
Precision: 1.00
Recall: 1.00
F1-Score: 1.00


In [ ]:
def generate_response(user_input):
    """
    Generate chatbot response based on semantic similarity.
    """
    # Encode masukan pengguna
    user_embedding = model.encode(user_input, convert_to_tensor=True)

    # Encode semua pertanyaan di dataset
    question_embeddings = model.encode(questions, convert_to_tensor=True)

    # Hitung kesamaan kosinus antara masukan pengguna dan pertanyaan dataset
    similarities = util.cos_sim(user_embedding, question_embeddings)

    # Cari pertanyaan dengan kesamaan tertinggi
    max_idx = similarities.argmax().item()
    max_similarity = similarities[0, max_idx].item()

    # Threshold untuk memastikan relevansi
    if max_similarity > 0.7:  # Anda bisa sesuaikan nilai threshold ini
        return responses[max_idx]
    else:
        return "Maaf, saya tidak memahami pertanyaan Anda. Bisa Anda jelaskan lebih lanjut?"


In [ ]:
print("Chatbot: Halo! Saya di sini untuk membantu Anda. Ketik 'keluar' untuk berhenti.")

while True:
    user_input = input("Anda: ")
    if user_input.lower() == "keluar":
        print("Chatbot: Terima kasih! Sampai jumpa.")
        break

    response = generate_response(user_input)
    print(f"Chatbot: {response}")


Chatbot: Halo! Saya di sini untuk membantu Anda. Ketik 'keluar' untuk berhenti.


KeyboardInterrupt: Interrupted by user